In [ ]:
from tkinter import * 
from tkinter import filedialog    
from tkinter import messagebox   
from IPython.display import Image
import pytesseract
import numpy as np
import cv2
root = Tk()


##function


def about():
    messagebox.showinfo("about","파일을 업로드 하시면 해당 이미지의 문자를 찾아보여줍니다!")

def upload():
    filename = filedialog.askopenfilename(initialdir="/", title="Select file",
                                          filetypes=(("jpg files", "*.jpg"),
                                          ("all files", "*.*")))
    large = cv2.imread(filename)
    small = cv2.cvtColor(large, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    grad = cv2.morphologyEx(small, cv2.MORPH_GRADIENT, kernel)
    _, bw = cv2.threshold(grad, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))
    connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)
# using RETR_EXTERNAL instead of RETR_CCOMP
    contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    mask = np.zeros(bw.shape, dtype=np.uint8)
    for idx in range(len(contours)):
        x, y, w, h = cv2.boundingRect(contours[idx])
        mask[y:y+h, x:x+w] = 0
        cv2.drawContours(mask, contours, idx, (255, 255, 255), -1)
        r = float(cv2.countNonZero(mask[y:y+h, x:x+w])) / (w * h)
        if r > 0.45 and w > 8 and h > 8:
            cv2.rectangle(large, (x, y), (x+w-1, y+h-1), (0, 255, 0), 2)
# show image with contours rect
    cv2.imshow('rects', large)
    cv2.waitKey()


    print(filename)
    result=pytesseract.image_to_string(filename, lang='kor')
    print(result)
    
## program info
root.title ("OCR TEST")
root.geometry("540x380") 

##label

label1 = Label (root, text= "원하는 이미지를 업로드 해주세요!")
label1.pack()


## button 
btn1= Button(root,text="파일 업로드",command=upload)

btn1.pack(side="bottom",anchor="w")



##Manu

menubar = Menu(root)  
helpmenu = Menu(menubar, tearoff=0)
menubar.add_cascade(label="Help", menu=helpmenu)
helpmenu.add_command(label="About...", command=about)

root.config(menu=menubar)             # 생성된 객체를 위에서 생성된 메뉴바에 연결   


##exec
root.mainloop()